In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
TRAIN_PATH = "/content/drive/My Drive/Colab Notebooks/FER/train.csv"
TEST_PATH = "/content/drive/My Drive/Colab Notebooks/FER/test.csv"
MODEL_PATH = "/content/drive/My Drive/Colab Notebooks/FER/trained_model.h5"

In [5]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
print(train_df.describe)
print(test_df.describe)

<bound method NDFrame.describe of        emotion                                             pixels
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1            0  151 150 147 155 148 133 111 140 170 174 182 15...
2            2  231 212 156 164 174 138 161 173 182 200 106 38...
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...        ...                                                ...
28704        2  84 85 85 85 85 85 85 85 86 86 86 87 86 86 91 9...
28705        0  114 112 113 113 111 111 112 113 115 113 114 11...
28706        4  74 81 87 89 95 100 98 93 105 120 127 133 146 1...
28707        0  222 227 203 90 86 90 84 77 94 87 99 119 134 14...
28708        4  195 199 205 206 205 203 206 209 208 210 212 21...

[28709 rows x 2 columns]>
<bound method NDFrame.describe of                                                  pixels
0     254 254 254 254 254 249 255 160 2 58 53 70 77 ...
1

In [6]:
enc_train = OneHotEncoder(handle_unknown='ignore')

train_labels = train_df['emotion'].to_numpy().reshape(-1,1)
enc_train.fit(train_labels)
train_labels = enc_train.transform(train_labels).toarray() 
n_samples = train_labels.shape[0]
train_image_data =  np.array([np.fromstring(sample, dtype=int, sep=' ').reshape(48,48,1) for sample in train_df['pixels'].to_numpy()]) 
train_image_data = train_image_data.astype(np.float32)/255.0


print(train_image_data.shape)
print(train_labels.shape)
print(n_samples)

#------------

test_n_samples = test_df.shape[0]
test_image_data =  np.array([np.fromstring(sample, dtype=int, sep=' ').reshape(48,48,1) for sample in test_df['pixels'].to_numpy()]) 

print(test_image_data.shape)
print(test_n_samples)

#-------------
train_datagen = ImageDataGenerator(
    rescale=1.0,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)

train_datagen.fit(train_image_data)
#-----------------
test_datagen = ImageDataGenerator(rescale=1.0)
test_datagen.fit(test_image_data)

(28709, 48, 48, 1)
(28709, 7)
28709
(7178, 48, 48, 1)
7178


In [0]:
EMOTIONS = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

In [8]:
from random import randint

rand_num = randint(0,n_samples)
print(rand_num)

# print("emotion {}".format(train_labels[rand_num]) )
# plt.imshow(train_image_data[rand_num])
# plt.show()

5371


In [0]:
#building keras model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.layers.advanced_activations import ELU
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers
from keras.regularizers import l1
from keras.callbacks import EarlyStopping

In [0]:
def baseline_model(drop_rate=0.1, regularization_val=0.0001):
  # Create the model
  model = Sequential()

  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                   kernel_regularizer=regularizers.l2(regularization_val), 
                   padding='same',
                   input_shape=(48,48,1)))
  # model.add(BatchNormalization())

  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',
                   kernel_regularizer=regularizers.l2(regularization_val), 
                   padding='same') )
  # model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', 
                   kernel_regularizer=regularizers.l2(regularization_val), 
                   padding='same'))
  # model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', 
                   kernel_regularizer=regularizers.l2(regularization_val), 
                   padding='same'))
  # model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', 
  #                  kernel_regularizer=regularizers.l2(regularization_val)))
  # model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', 
  #                  kernel_regularizer=regularizers.l2(regularization_val)))
  # model.add(MaxPooling2D(pool_size=(2, 2)))


  model.add(Conv2D(7, kernel_size=(1, 1), activation='relu', 
                   kernel_regularizer=regularizers.l2(regularization_val), 
                   padding='same'))
  # model.add(BatchNormalization())

  model.add(Conv2D(7, kernel_size=(4, 4), activation='relu', 
                   kernel_regularizer=regularizers.l2(regularization_val), 
                   padding='same'))
  # model.add(BatchNormalization())

  model.add(Flatten())

  # model.add(Dense(4096, activation='relu'))
  # model.add(Dropout(drop_rate))
  # model.add(Dense(4096, activation='relu'))
  # model.add(Dropout(drop_rate))

  model.add(Dense(7, activation='relu'))

  model.add(Activation("softmax"))

  model.summary()

  adam = Adam(lr = 1e-3)
  model.compile(loss='categorical_crossentropy', optimizer=adam)
  return model



In [11]:
model_ = baseline_model(drop_rate=0.1, regularization_val=0.0001)






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 128)       147584    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 128)    

In [0]:
es = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
# history = model_.fit(train_image_data, train_labels, validation_split=0.20, epochs=50, batch_size=64, verbose=1, callbacks=[es])
batch_size=64
history = model_.fit_generator(
    train_datagen.flow(train_image_data,train_labels, batch_size=batch_size, subset='training'), 
    epochs=500, 
    steps_per_epoch=len(train_image_data) / batch_size, 
    verbose=1,
    validation_data=train_datagen.flow(train_image_data,train_labels, batch_size=batch_size, subset='validation'),
    callbacks=[es])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/500





449/448 [==============================] - 27s 60ms/step - loss: 1.8271 - val_loss: 1.8007
Epoch 2/500
449/448 [==============================] - 13s 28ms/step - loss: 1.7582 - val_loss: 1.7268
Epoch 3/500
449/448 [==============================] - 13s 28ms/step - loss: 1.7108 - val_loss: 1.6811
Epoch 4/500
449/448 [==============================] - 13s 28ms/step - loss: 1.6597 - val_loss: 1.6528
Epoch 5/500
449/448 [==============================] - 13s 28ms/step - loss: 1.5968 - val_loss: 1.5913
Epoch 6/500
449/448 [==============================] - 12s 28ms/step - loss: 1.5508 - val_loss: 1.5244
Epoch 7/500
449/448 [==============================] - 13s 29ms/step - loss: 1.5202 - val_loss: 1.5051
Epoch 8/500
449/448 [==============================] - 13s 28ms/step - loss: 1.4810 - val_loss: 1.4826
Epoch 9/500
449/448 [==============================] - 12s 28ms/step - los

In [0]:

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
model_.save(MODEL_PATH)